ALS + GB
Подобрать другие retrivials

In [1]:
%autosave 60
%reload_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "..")  # noqa: E702

Autosaving every 60 seconds


In [2]:
from catboost.utils import get_gpu_device_count

get_gpu_device_count()

1

In [3]:
import implicit.gpu
import polars as pl
import numpy as np
import implicit
from scipy.sparse import csr_matrix
from pathlib import Path
from catboost import CatBoostRanker, Pool
from datetime import timedelta


implicit.gpu.HAS_CUDA
from tools.retrievers import PopularItemsRetriever, CFRetriever

/home/laptopml/avito_cup_ml_2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pathlib import Path

DATA_DIR = Path.cwd().parent / "data" / "avito_ml_cup"
# Разделение на три части: тренировочную, валидационную и тестовую
VALID_DAYS_RETRIEVER = 7
VALID_DAYS_RERANKER = 7

In [5]:
df_test_users = pl.read_parquet(f"{DATA_DIR}/test_users.pq")
df_clickstream = pl.read_parquet(f"{DATA_DIR}/clickstream.pq")

df_cat_features = pl.read_parquet(f"{DATA_DIR}/cat_features.pq")
df_text_features = pl.read_parquet(f"{DATA_DIR}/text_features.pq")
df_event = pl.read_parquet(f"{DATA_DIR}/events.pq")

In [6]:
df_clickstream

cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,4247649,17,2025-02-17 14:55:17,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458
…,…,…,…,…,…,…
149999,4999183,17,2025-01-20 12:23:47,2,2,71511
149999,25999164,17,2025-01-24 14:26:57,2,2,71514
149999,12138732,17,2025-02-12 13:11:42,2,2,51162


In [7]:
df_clickstream["event_date"].max() - df_clickstream["event_date"].min()

datetime.timedelta(days=44)

In [8]:
# Определяем границы для разделения данных
max_date = df_clickstream["event_date"].max()
retrivial_threshold = max_date - timedelta(days=VALID_DAYS_RETRIEVER + VALID_DAYS_RERANKER)
reranker_threshold = max_date - timedelta(days=VALID_DAYS_RERANKER)

# Разделяем данные
df_train_retrivial = df_clickstream.filter(pl.col("event_date") < retrivial_threshold)
df_train_reranker = df_clickstream.filter(
    pl.col("event_date").is_between(retrivial_threshold, reranker_threshold)
)
df_valid = df_clickstream.filter(pl.col("event_date") > reranker_threshold)


# Создаем оценочные наборы для валидации и тестирования
def prepare_eval_set(df_eval_raw, df_train_data):
    # Выбираем только нужные колонки
    df_eval = df_eval_raw[["cookie", "node", "event"]]

    # Исключаем пары cookie-node, которые уже встречались в тренировочных данных
    df_eval = df_eval.join(df_train_data, on=["cookie", "node"], how="anti")

    # Оставляем только контактные события
    df_eval = df_eval.filter(
        pl.col("event").is_in(
            df_event.filter(pl.col("is_contact") == 1)["event"].unique()
        )
    )

    # Оставляем только существующие в тренировочных данных cookies и nodes
    df_eval = df_eval.filter(
        pl.col("cookie").is_in(df_train_data["cookie"].unique())
    ).filter(pl.col("node").is_in(df_train_data["node"].unique()))

    # Оставляем только уникальные пары cookie-node
    df_eval = df_eval.unique(["cookie", "node"])

    return df_eval


# Подготавливаем валидационный и тестовый наборы
df_valid_eval_retrivial = prepare_eval_set(df_train_reranker, df_train_retrivial)
df_valid_eval_reranker = prepare_eval_set(
    df_valid, pl.concat([df_train_retrivial, df_train_reranker])
)

df_train_retrivial = df_train_retrivial.join(df_event, on="event")

df_train_reranker = df_train_reranker.join(df_event, on="event")

df_valid = df_valid.join(df_event, on="event")

/tmp/ipykernel_82092/655004105.py:23: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_82092/655004105.py:30: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  df_eval = df_eval.filter(
/tmp/ipykernel_82092/655004105.py:32: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  ).filter(pl.col("node").is_in(df_train_data["node"].unique()))


In [9]:
df_valid_eval_reranker.join(df_event, on="event").sort("cookie")

cookie,node,event,is_contact
i64,u32,i64,i64
0,130589,19,1
2,152695,5,1
2,163823,10,1
10,51162,10,1
22,1906,5,1
…,…,…,…
149975,238794,5,1
149975,234860,10,1
149980,1911,10,1


In [10]:
# Выводим информацию о размерах наборов данных
print(f"Размер тренировочного набора retrivial: {df_train_retrivial.shape}")
print(f"Размер валидационного набора reranker: {df_train_reranker.shape}")
print(f"Размер валидационного набора: {df_valid.shape}")
print(
    f"Размер валидационного набора для оценки retrivial: {df_valid_eval_retrivial.shape}"
)
print(
    f"Размер валидационного набора для оценки reranker: {df_valid_eval_reranker.shape}"
)

Размер тренировочного набора retrivial: (45631757, 8)
Размер валидационного набора reranker: (11400304, 8)
Размер валидационного набора: (11774091, 8)
Размер валидационного набора для оценки retrivial: (72271, 3)
Размер валидационного набора для оценки reranker: (74771, 3)


# Train model

## Train ALS

In [11]:
class ALSRetriever:
    def __init__(self, **params_als):
        self.model = implicit.als.AlternatingLeastSquares(**params_als)

        self._user_id_to_index: dict[int, int] | None = None
        self._item_id_to_index: dict[int, int] | None = None
        self._index_to_item_id: dict[int, int] | None = None
        self._sparse_matrix = None

    def _create_mappings(self, user_ids, item_ids):
        """Создает маппинги между ID и индексами для пользователей и товаров"""
        self._user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
        self._item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
        self._index_to_item_id = {v: k for k, v in self._item_id_to_index.items()}
        self._index_to_user_id = {v: k for k, v in self._user_id_to_index.items()}

    def fit(
        self,
        users,
        nodes,
        events=None,
        event_weights=None,
        contact_event_boost=10.0,
        contact_events=None,
        is_contact_col=None,
    ):
        """
        Обучает модель ALS

        Args:
            users (pl.Series): ID пользователей
            nodes (pl.Series): ID элементов (товаров)
            events (pl.Series, optional): Типы событий
            event_weights (dict, optional): Словарь весов для разных типов событий
            contact_event_boost (float): Множитель для контактных событий
            contact_events (list, optional): Список ID контактных событий
            is_contact_col (pl.Series, optional): Колонка с флагом контакта

        Returns:
            self: Возвращает self для возможности цепочки вызовов
        """
        user_ids = users.unique().to_list()
        item_ids = nodes.unique().to_list()

        self._create_mappings(user_ids, item_ids)

        rows = users.replace_strict(self._user_id_to_index).to_list()
        cols = nodes.replace_strict(self._item_id_to_index).to_list()

        # Определение весов взаимодействий
        if events is not None and event_weights:
            values = [event_weights.get(event, 1.0) for event in events.to_list()]

            # Если указаны контактные события, увеличиваем их вес
            if contact_events:
                values = [
                    val * contact_event_boost if events[i] in contact_events else val
                    for i, val in enumerate(values)
                ]
        elif is_contact_col is not None:
            # Если есть прямой признак контакта, используем его для весов
            values = is_contact_col.to_list()
        else:
            # По умолчанию все взаимодействия имеют вес 1
            values = [1] * len(users)

        # Создаем разреженную матрицу взаимодействий
        self._sparse_matrix = csr_matrix(
            (values, (rows, cols)), shape=(len(user_ids), len(item_ids))
        )

        # Обучаем модель
        self.model.fit(self._sparse_matrix)

    def recommend(self, user_ids, n=40, filter_already_liked_items=True):
        """
        Генерирует рекомендации для указанных пользователей

        Args:
            user_ids (list): Список ID пользователей для рекомендаций
            n (int): Количество рекомендаций для каждого пользователя
            filter_already_liked_items (bool): Исключать ли уже взаимодействовавшие товары
            recalculate_user (bool): Пересчитывать ли модель для пользователя

        Returns:
            pl.DataFrame: DataFrame с рекомендациями
        """
        if not self._user_id_to_index:
            raise ValueError("Сначала надо fit.")
        # Преобразуем ID пользователей в индексы
        user_indices = []
        valid_users = []

        for user_id in user_ids:
            if user_id in self._user_id_to_index:
                user_indices.append(self._user_id_to_index[user_id])  # type: ignore
                valid_users.append(user_id)

        if not user_indices:
            return pl.DataFrame({"cookie": [], "node": [], "scores": []})

        # Получаем рекомендации
        user_index_array = np.array(user_indices)
        recommendations, scores = self.model.recommend(
            user_index_array,
            self._sparse_matrix[user_index_array],  # type: ignore
            N=n,
            filter_already_liked_items=filter_already_liked_items,
        )

        # Преобразуем индексы товаров обратно в ID
        node_lists = [
            [self._index_to_item_id[idx] for idx in rec] for rec in recommendations
        ]  # type: ignore

        # Создаем DataFrame с результатами
        df_pred = pl.DataFrame(
            {"node": node_lists, "cookie": valid_users, "scores": scores.tolist()}
        )

        # Разворачиваем списки
        df_pred = df_pred.explode(["node", "scores"])

        return df_pred

    def save_model(self, path: str):
        import pickle

        model_data = {
            "model": self.model,
            "user_id_to_index": self._user_id_to_index,
            "item_id_to_index": self._item_id_to_index,
            "index_to_item_id": self._index_to_item_id,
            "index_to_user_id": self._index_to_user_id,
            "sparse_matrix": self._sparse_matrix,
        }

        with open(path, "wb") as f:
            pickle.dump(model_data, f)

    @classmethod
    def load(cls, path):
        """
        Загружает модель из файла

        Args:
            path (str): Путь к файлу модели

        Returns:
            ALSRetrivier: Загруженная модель
        """
        import pickle

        with open(path, "rb") as f:
            model_data = pickle.load(f)

        retriever = cls()
        retriever.model = model_data["model"]
        retriever._user_id_to_index = model_data["user_id_to_index"]
        retriever._item_id_to_index = model_data["item_id_to_index"]
        retriever._index_to_item_id = model_data["index_to_item_id"]
        retriever._index_to_user_id = model_data["index_to_user_id"]
        retriever._sparse_matrix = model_data["sparse_matrix"]

        return retriever


def get_als_pred(
    users,
    nodes,
    events,
    user_to_pred,
    event_weights=None,
    generate_candidates: int = 40,
):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()

    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v: k for k, v in item_id_to_index.items()}

    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()

    # Используем веса событий или 1 по умолчанию
    if event_weights is not None and events is not None:
        values = [event_weights.get(event, 1.0) for event in events.to_list()]
    else:
        values = [1] * len(users)

    sparse_matrix = csr_matrix(
        (values, (rows, cols)), shape=(len(user_ids), len(item_ids))
    )

    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=100)
    model.fit(
        sparse_matrix,
    )

    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])

    recommendations, scores = model.recommend(
        user4pred,
        sparse_matrix[user4pred],
        N=generate_candidates,
        filter_already_liked_items=True,
    )

    df_pred = pl.DataFrame(
        {
            "node": [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ],
            "cookie": list(user_to_pred),
            "scores": scores.tolist(),
        }
    )
    df_pred = df_pred.explode(["node", "scores"])
    return df_pred

In [12]:
# als_retriever_1 = ALSRetriever(iterations=50, factors=250, regularization=0.3, use_gpu=True)

als_retriever = ALSRetriever(iterations=30, factors=100, regularization=0.7, use_gpu=True)

In [13]:
def clean_interactions_for_als(df_clickstream):
    """
    Комплексная очистка для ALS
    """
    # 1. Агрегируем по cookie-node с приоритетом контактных событий
    cleaned = df_clickstream.group_by(["cookie", "node", "item"]).agg([
        pl.col("is_contact").max().alias("is_contact"),  # Если хоть раз был контакт
        pl.col("event_date").max().alias("last_interaction"),
        pl.count().alias("total_interactions"),
        pl.col("is_contact").sum().alias("contact_interactions")
    ]).with_columns([
        # Создаем финальный вес для ALS
        pl.when(pl.col("is_contact") == 1)
        .then(5.0 + pl.col("contact_interactions").log1p())  # Больше контактов = больше уверенности
        .otherwise(0.5 + pl.col("total_interactions").log1p() * 0.1)  # Неконтактные получают малый вес
        .alias("als_weight")
    ])
    
    return cleaned.select(["cookie", "node", "is_contact", "als_weight"])


df_clean_retrivial = clean_interactions_for_als(df_train_retrivial)

/tmp/ipykernel_82092/1562741574.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("total_interactions"),


In [14]:
users = df_clean_retrivial["cookie"]
nodes = df_clean_retrivial["node"]
weights_for_als = df_clean_retrivial["als_weight"]
eval_users = df_valid_eval_retrivial["cookie"].unique().to_list()

In [15]:
def create_event_weights(clickstream_df: pl.DataFrame):
    event_count = {
        row[0]: row[1] for row in clickstream_df.group_by("event").len().rows()
    }

    return {event_id: 1.0 / np.log1p(count) for event_id, count in event_count.items()}


# inverse_freq_weights = create_event_weights(df_train_retrivial.select("event"))

# def create_frequency_weights(df_clickstream, base_contact_weight=10.0):
#     """
#     Веса на основе частоты событий - редкие события получают больший вес
#     """
#     event_counts = df_clickstream.group_by("event").agg([
#         pl.count().alias("event_count"),
#         pl.col("is_contact").first().alias("is_contact")
#     ])
    
#     max_count = event_counts["event_count"].max()
#     weights = {}
    
#     for row in event_counts.rows():
#         event_id, count, is_contact = row
#         if is_contact == 1:
#             # Для контактных: обратно пропорционально частоте
#             weights[event_id] = base_contact_weight * (np.log1p(max_count) / np.log1p(count))
#         else:
#             # Для неконтактных: небольшой базовый вес
#             weights[event_id] = 0.1
    
#     return weights

# frequency_weights = create_frequency_weights(df_train_retrivial, 1)

In [16]:
als_retriever.fit(
    users, nodes, is_contact_col=weights_for_als)

100%|██████████| 30/30 [00:32<00:00,  1.10s/it]


In [17]:
df_pred_alt = als_retriever.recommend(eval_users, 200)

In [18]:
del als_retriever

In [ ]:
# from hydra import compose, initialize
# def config_retriever(config_path, config_name):
#     with initialize(config_path=config_path, version_base=None):
#         cfg = compose(config_name=config_name)
#         return cfg

In [ ]:
# config_retrievers = config_retriever("../configs/retrieval_models", "popular")

# popul_items_retriever = PopularItemsRetriever(config_retrievers)
# popul_items_retriever.set_data(df_train_retrivial, df_cat_features)

# popul_nodes = popul_items_retriever.recommend(eval_users)

In [ ]:
# popul_nodes.sort("score_popular_items_1d")

cookie,node,score_popular_items_1d,score_popular_items_30d,score_popular_items_by_category_7d,score_popular_items_by_category_30d
i64,i64,f64,f64,f64,f64
0,13939,null,null,1.0,1.0
0,10893,null,null,1.0,1.0
0,153578,null,null,1.0,1.0
0,27,null,null,1.0,1.0
0,57915,null,null,1.0,1.0
…,…,…,…,…,…
149967,151453,1.0,1.0,1.0,1.0
149973,151453,1.0,1.0,1.0,1.0
149975,151453,1.0,1.0,1.0,1.0


In [19]:
class PopularItemsFeatureExtractor:
    """
    Класс для получения популярных товаров по различным критериям
    """

    def __init__(self, clickstream_df=None, categories_df=None):
        """
        Инициализирует ретривер популярных товаров

        Args:
            clickstream_df (pl.DataFrame, optional): DataFrame с данными кликстрима
            categories_df (pl.DataFrame, optional): DataFrame с категориями товаров
        """
        self.clickstream_df = clickstream_df
        self.categories_df = categories_df

        # Кэш для хранения результатов
        self._cache = {}

    def set_data(self, clickstream_df, categories_df=None):
        """
        Устанавливает данные для работы

        Args:
            clickstream_df (pl.DataFrame): DataFrame с данными кликстрима
            categories_df (pl.DataFrame, optional): DataFrame с категориями товаров

        Returns:
            self: Возвращает self для цепочки вызовов
        """
        self.clickstream_df = clickstream_df
        if categories_df is not None:
            self.categories_df = categories_df

        # Сбрасываем кэш при обновлении данных
        self._cache = {}

        return self

    def get_popular_nodes(self, days=7, use_cache=True):
        """
        Получает популярные товары за указанный период

        Args:
            days (int): Количество дней для анализа
            top_k (int): Количество лучших товаров
            use_cache (bool): Использовать кэш или пересчитать

        Returns:
            pl.DataFrame: Таблица с популярными товарами и количеством взаимодействий
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        cache_key = f"popular_{days}"
        if use_cache and cache_key in self._cache:
            return self._cache[cache_key]

        # Определяем временное окно
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Подсчитываем популярность каждого node
        popular_nodes = (
            recent_data.group_by("node")
            .agg(pl.count().log1p().alias(f"popularity_last_{days}_days"))
        )

        # Кэшируем результаты
        if use_cache:
            self._cache[cache_key] = popular_nodes

        return popular_nodes

    def get_popular_nodes_by_category(self, days=7, use_cache=True):
        """
        Получает популярные товары по категориям за указанный период

        Args:
            days (int): Количество дней для анализа
            top_k (int): Количество лучших товаров на категорию
            use_cache (bool): Использовать кэш или пересчитать

        Returns:
            pl.DataFrame: Таблица с популярными товарами по категориям
        """
        if self.clickstream_df is None or self.categories_df is None:
            raise ValueError("Не установлены данные кликстрима или категорий товаров")

        cache_key = f"popular_by_category_{days}"
        if use_cache and cache_key in self._cache:
            return self._cache[cache_key]

        # Определяем временное окно
        cutoff_date = self.clickstream_df["event_date"].max() - timedelta(days=days)
        recent_data = self.clickstream_df.filter(pl.col("event_date") > cutoff_date)

        # Соединяем с категориями
        recent_with_cat = recent_data.join(
            self.categories_df[["node", "category"]].unique(), on="node", how="inner"
        )

        # Подсчитываем популярность по категориям
        popular_by_cat = (
            recent_with_cat.group_by(["category", "node"])
            .agg(pl.count().log1p().alias(f"category_popularity_last_{days}_days"))
        )

        # Кэшируем результаты
        if use_cache:
            self._cache[cache_key] = popular_by_cat

        return popular_by_cat[["node", f"category_popularity_last_{days}_days"]]

    def get_trending_items(self, days_window=3, previous_days=7):
        """
        Получает тренды - товары, растущие по популярности

        Args:
            days_window (int): Размер окна для текущей популярности (дни)
            previous_days (int): Количество дней для сравнения с предыдущим периодом
            top_k (int): Количество растущих товаров для возврата

        Returns:
            pl.DataFrame: Таблица с растущими товарами
        """
        if self.clickstream_df is None:
            raise ValueError("Не установлены данные кликстрима")

        max_date = self.clickstream_df["event_date"].max()

        # Данные для текущего окна
        current_cutoff = max_date - timedelta(days=days_window)
        current_data = self.clickstream_df.filter(pl.col("event_date") > current_cutoff)

        # Данные для предыдущего окна
        prev_start = current_cutoff - timedelta(days=previous_days)
        prev_data = self.clickstream_df.filter(
            (pl.col("event_date") <= current_cutoff)
            & (pl.col("event_date") > prev_start)
        )

        # Вычисляем популярность для текущего и предыдущего окна
        current_pop = current_data.group_by("node").agg(
            pl.count().alias("current_count")
        ).cast(pl.Int32)

        prev_pop = prev_data.group_by("node").agg(pl.count().alias("previous_count")).cast(pl.Int32)

        # Объединяем и вычисляем рост
        trending = (
            current_pop.join(prev_pop, on="node", how="left")
            .with_columns(
                [
                    pl.col("previous_count").fill_null(0),
                    (pl.col("current_count") - pl.col("previous_count")).alias(
                        "growth"
                    ),
                    (
                        (pl.col("current_count") - pl.col("previous_count"))
                        / (pl.col("previous_count") + 1)
                    ).alias(f"trending_items_pd_{previous_days}_wd_{days_window}"),
                ]
            )
        )

        return trending[["node", f"trending_items_pd_{previous_days}_wd_{days_window}"]]

In [20]:
popular_extract = PopularItemsFeatureExtractor(df_train_retrivial, df_cat_features)

# Получаем популярные товары за последнюю неделю и месяц
popular_count_week = popular_extract.get_popular_nodes(days=7)
popular_count_month = popular_extract.get_popular_nodes(days=30)
# popular_count_by_category_week = popular_extract.get_popular_nodes_by_category(days=7)
# popular_count_by_category_month = popular_extract.get_popular_nodes_by_category(days=30)
trending_count_week = popular_extract.get_trending_items(days_window=3, previous_days=7)

/tmp/ipykernel_82092/2869826841.py:66: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().log1p().alias(f"popularity_last_{days}_days"))
/tmp/ipykernel_82092/2869826841.py:145: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("current_count")
/tmp/ipykernel_82092/2869826841.py:148: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  prev_pop = prev_data.group_by("node").agg(pl.count().alias("previous_count")).cast(pl.Int32)


# Двустадийная система рекомендаций
## Этап 1: ALS для первичного отбора релевантных товаров
## Этап 2: CatBoost для ранжирования и уточнения результатов

In [21]:
df_train_reranker

cookie,item,event,event_date,platform,surface,node,is_contact
i64,i64,i64,datetime[ns],i64,i64,u32,i64
1,6830678,17,2025-02-09 13:17:03,2,2,15,0
1,6830678,17,2025-02-09 13:16:58,2,2,15,0
1,14233689,17,2025-02-09 16:53:28,2,2,214458,0
1,7928899,17,2025-02-15 13:44:59,2,11,214231,0
1,18436084,17,2025-02-14 15:14:23,2,11,214458,0
…,…,…,…,…,…,…,…
149989,14257821,17,2025-02-12 12:42:11,2,2,152108,0
149995,16818719,17,2025-02-14 05:59:18,3,11,115733,0
149996,4516135,17,2025-02-09 04:38:46,2,2,122326,0


In [49]:
# df_retrievers = df_pred_alt.join(
#     popul_nodes,
#     on=["cookie", "node"],
#     how="full",
#     coalesce=True
# )

In [50]:
# df_retrievers

In [22]:
df_candidates_train = df_pred_alt.join(
    df_train_reranker["node", "cookie", "is_contact"]
    .sort("is_contact", descending=True)
    .unique(["cookie", "node"]),
    on=["cookie", "node"],
    how="left",
).fill_null(0)

In [23]:
# Подготовка данных для CatBoost
def prepare_features_for_boosting(
    df_retrievals,
    df_clickstream,
    df_cat_features,
    **kwargs
):
    """Подготавливает признаки для обучения бустинга на основе результатов ALS"""
    # Добавляем информацию о рейтинге ALS
    features = df_retrievals.rename({"scores": "als_score"})

    features = features.join(
        df_cat_features.select(["node", "category"]).unique(),
        on="node",
        how="left",
    ).fill_null(-1)

    for item_features in kwargs.values():
        if "node" in item_features.columns:
            if "category" in item_features.columns:
                features = features.join(
                    item_features,
                    on=["node", "category"],
                    how="left",
                ).fill_null(0)
            else:
                features = features.join(
                    item_features,
                    on=["node"],
                    how="left"
                ).fill_null(0)

    # Добавляем количество взаимодействий пользователя
    user_features = df_clickstream.group_by("cookie").agg(
        [
            pl.len().alias("user_total_interactions"),
            pl.n_unique("node").alias("user_unique_nodes"),
            (pl.col("is_contact").sum() / pl.count()).alias("user_contact_ratio"),
        ]
    )

    features = features.join(user_features, on="cookie", how="left")
 
    recent_interactions = df_clickstream.filter(
        pl.col("event_date") > (df_clickstream["event_date"].max() - timedelta(days=7))
    )
    recent_user_features = recent_interactions.group_by("cookie").agg(
        pl.count().alias("user_recent_interactions")
    )
    features = features.join(recent_user_features, on="cookie", how="left")

    return features.fill_null(0)

# Готовим данные для CatBoost
train_data_for_boosting = prepare_features_for_boosting(
    df_candidates_train,
    df_train_retrivial,
    df_cat_features,
    popular_count_week=popular_count_week,
    popular_count_month=popular_count_month,
    trending_count_week=trending_count_week,
)

/tmp/ipykernel_82092/587475119.py:38: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.col("is_contact").sum() / pl.count()).alias("user_contact_ratio"),
/tmp/ipykernel_82092/587475119.py:48: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("user_recent_interactions")


In [24]:
train_data_for_boosting

node,cookie,als_score,is_contact,category,popularity_last_7_days,popularity_last_30_days,trending_items_pd_7_wd_3,user_total_interactions,user_unique_nodes,user_contact_ratio,user_recent_interactions
i64,i64,f64,i64,i64,f64,f64,f64,u32,u32,f64,u32
115834,0,0.819912,0,19,9.08659,10.52183,-0.580346,225,73,0.066667,107
214235,0,0.743648,0,37,8.758884,10.163965,-0.553094,225,73,0.066667,107
214234,0,0.732507,0,37,9.208238,10.627309,-0.578846,225,73,0.066667,107
115713,0,0.72798,0,19,8.987197,10.470703,-0.610667,225,73,0.066667,107
1923,0,0.688839,0,61,9.986495,11.357627,-0.561453,225,73,0.066667,107
…,…,…,…,…,…,…,…,…,…,…,…
130608,149998,0.079378,0,31,7.737616,9.087381,-0.518519,308,62,0.012987,20
136771,149998,0.079257,0,24,7.984803,9.344696,-0.496238,308,62,0.012987,20
214348,149998,0.079091,0,37,6.57368,7.975908,-0.595683,308,62,0.012987,20


In [25]:
train_data_for_boosting.columns

['node',
 'cookie',
 'als_score',
 'is_contact',
 'category',
 'popularity_last_7_days',
 'popularity_last_30_days',
 'trending_items_pd_7_wd_3',
 'user_total_interactions',
 'user_unique_nodes',
 'user_contact_ratio',
 'user_recent_interactions']

In [26]:
feature_cols = [
    'node',
    'category',
    'als_score',
    # 'score_popular_items_1d',
    # 'score_popular_items_30d',
    # 'score_popular_items_by_category_7d',
    # 'score_popular_items_by_category_30d',
    'popularity_last_7_days',
    'popularity_last_30_days',
    'trending_items_pd_7_wd_3',
    'user_total_interactions',
    'user_unique_nodes',
    'user_contact_ratio',
    'user_recent_interactions'
]

cat_cols = ["category"]

target_col = "is_contact"

In [27]:
model = CatBoostRanker(
    boosting_type="Plain",
    iterations=300,
    learning_rate=0.04,
    depth=8,
    objective="PairLogitPairwise:max_pairs=50000",
    random_seed=42,
    eval_metric="RecallAt:top=40",
    verbose=50,
    nan_mode="Min",
    task_type="GPU"
)

In [28]:
train_data_boosting = train_data_for_boosting.sample(fraction=0.8, seed=123).sort("cookie")

valid_data_boosting = train_data_for_boosting.join(train_data_boosting, on=train_data_boosting.columns, how="anti").sort("cookie")

In [29]:
train_data_boosting = Pool(train_data_boosting.select(feature_cols).to_pandas(), train_data_boosting.select(target_col).to_pandas(), cat_features=cat_cols, group_id=train_data_boosting["cookie"].to_list())

valid_data_boosting = Pool(valid_data_boosting.select(feature_cols).to_pandas(), valid_data_boosting.select(target_col).to_pandas(), cat_features=cat_cols, group_id=valid_data_boosting["cookie"].to_list())

In [30]:
model.fit(
    train_data_boosting,
    eval_set=valid_data_boosting
)

Groupwise loss function. OneHotMaxSize set to 10


Default metric period is 5 because RecallAt is/are not implemented for GPU
Metric RecallAt:top=40 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7947049	test: 0.9938499	best: 0.9938499 (0)	total: 944ms	remaining: 4m 42s
50:	learn: 0.8256061	test: 0.9980869	best: 0.9981017 (40)	total: 29.9s	remaining: 2m 25s
100:	learn: 0.8302167	test: 0.9982770	best: 0.9983510 (90)	total: 57.8s	remaining: 1m 53s
150:	learn: 0.8357528	test: 0.9983954	best: 0.9984251 (135)	total: 1m 26s	remaining: 1m 25s
200:	learn: 0.8396643	test: 0.9984794	best: 0.9985534 (195)	total: 1m 54s	remaining: 56.6s
250:	learn: 0.8443489	test: 0.9985287	best: 0.9985831 (220)	total: 2m 23s	remaining: 28s
299:	learn: 0.8475912	test: 0.9985436	best: 0.9986127 (280)	total: 2m 51s	remaining: 0us
bestTest = 0.9986126745
bestIteration = 280
Shrink model to first 281 iterations.


# Оценка моделей
## Метрика Recall@K

In [35]:
def recall_at(df_true, df_pred, k=40):
    """Вычисляет метрику Recall@K для рекомендаций"""
    return (
        df_true[["node", "cookie"]]
        .join(
            df_pred.group_by("cookie")
            .head(k)
            .with_columns(value=1)[["node", "cookie", "value"]],
            how="left",
            on=["cookie", "node"],
        )
        .select([pl.col("value").fill_null(0), "cookie"])
        .group_by("cookie")
        .agg([pl.col("value").sum() / pl.col("value").count()])["value"]
        .mean()
    )


def evaluate_models(
    df_true, df_als_pred, df_catboost_pred=None, k_values=[10, 20, 30, 40]
):
    """Сравнивает модели по метрике Recall@K для разных значений K"""
    results = {}

    for k in k_values:
        results[f"ALS_recall@{k}"] = recall_at(df_true, df_als_pred, k=k)
        if df_catboost_pred is not None:
            results[f"CatBoost_recall@{k}"] = recall_at(df_true, df_catboost_pred, k=k)

    return results


# Оцениваем модель ALS на валидационном наборе
als_results = evaluate_models(
    df_valid_eval_retrivial, df_pred_alt, k_values=[10, 20, 30, 40, 50, 60, 70, 100]
)
print("\nРезультаты ALS модели на валидационном наборе:")
for metric, value in als_results.items():
    print(f"{metric}: {value:.4f}")



Результаты ALS модели на валидационном наборе:
ALS_recall@10: 0.0655
ALS_recall@20: 0.1041
ALS_recall@30: 0.1319
ALS_recall@40: 0.1535
ALS_recall@50: 0.1733
ALS_recall@60: 0.1903
ALS_recall@70: 0.2056
ALS_recall@100: 0.2436


In [36]:
# recall_at(df_eval, df_pred_alt, k=40)

# Можно сохранить обученные модели и результаты для дальнейшего использования

# Пример организации модулей для ClearML:
"""
from tools.experiment_tracker import ExperimentTracker

# Инициализация трекера экспериментов
tracker = ExperimentTracker(project_name="avito_cup_ml_2025", task_name="als_catboost_pipeline")

# Логирование параметров
tracker.log_params({
    'als_factors': 60,
    'als_iterations': 10,
    'catboost_iterations': 500,
    'catboost_learning_rate': 0.05,
    'catboost_depth': 6
})

# Логирование метрик
tracker.log_metrics(als_results)

# Сохранение моделей
tracker.save_model(model, 'catboost_ranker')

# Завершение эксперимента
tracker.close()
"""


df_train_retriever_and_reranker = pl.concat([df_train_retrivial, df_train_reranker])

df_clean_retrivial_and_reranker = clean_interactions_for_als(df_train_retriever_and_reranker)

# Применение двустадийной модели на тестовом наборе
# Сначала получаем рекомендации от ALS

if not df_valid_eval_reranker.is_empty():
    test_users = df_valid_eval_reranker["cookie"].unique().to_list()
else:
    # Для тестовых пользователей
    test_users = df_test_users["cookie"].unique().to_list()


# Объединяем тренировочные и валидационные данные для финального обучения
full_train_users = df_clean_retrivial_and_reranker["cookie"]
full_train_nodes = df_clean_retrivial_and_reranker["node"]
full_weight_for_als = df_clean_retrivial_and_reranker["als_weight"]

# Создаем веса на основе обратной частоты и контактности
# inverse_freq_weights = create_event_weights(df_train_retriever_and_reranker.select("event"))

/tmp/ipykernel_82092/1562741574.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("total_interactions"),


In [37]:
als_retriever = ALSRetriever(iterations=30, factors=100, regularization=0.7)

als_retriever.fit(full_train_users, full_train_nodes, is_contact_col=full_weight_for_als)

# Получаем предсказания ALS для тестового набора
test_als_pred = als_retriever.recommend(test_users, 200)

100%|██████████| 30/30 [00:36<00:00,  1.21s/it]


In [ ]:
# popul_items_retriever.set_data(df_train_retriever_and_reranker, df_cat_features)

# popul_nodes_test = popul_items_retriever.recommend(test_users)

In [ ]:
# df_test_retrievers = test_als_pred.join(
#     popul_nodes_test,
#     on=["cookie", "node"],
#     how="full",
#     coalesce=True
# ).fill_null(0)

In [38]:
popular_extract = PopularItemsFeatureExtractor(df_train_retriever_and_reranker, df_cat_features)

# Получаем популярные товары за последнюю неделю и месяц
popular_count_week = popular_extract.get_popular_nodes(days=7)
popular_count_month = popular_extract.get_popular_nodes(days=30)
trending_count_week = popular_extract.get_trending_items(days_window=3, previous_days=7)

/tmp/ipykernel_82092/2869826841.py:66: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().log1p().alias(f"popularity_last_{days}_days"))
/tmp/ipykernel_82092/2869826841.py:145: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("current_count")
/tmp/ipykernel_82092/2869826841.py:148: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  prev_pop = prev_data.group_by("node").agg(pl.count().alias("previous_count")).cast(pl.Int32)


In [39]:
test_data_for_boosting = prepare_features_for_boosting(
    test_als_pred,
    df_train_retriever_and_reranker,
    df_cat_features,
    popular_count_week=popular_count_week,
    popular_count_month=popular_count_month,
    trending_count_week=trending_count_week,
).sort("cookie")

/tmp/ipykernel_82092/587475119.py:38: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.col("is_contact").sum() / pl.count()).alias("user_contact_ratio"),
/tmp/ipykernel_82092/587475119.py:48: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("user_recent_interactions")


In [40]:
test_data_for_boosting

node,cookie,als_score,category,popularity_last_7_days,popularity_last_30_days,trending_items_pd_7_wd_3,user_total_interactions,user_unique_nodes,user_contact_ratio,user_recent_interactions
i64,i64,f64,i64,f64,f64,f64,u32,u32,f64,u32
115834,0,0.847133,19,8.987822,10.519538,-0.623267,258,83,0.062016,33
214235,0,0.746747,37,8.664406,10.163503,-0.636349,258,83,0.062016,33
214234,0,0.712815,37,9.253974,10.640364,-0.585779,258,83,0.062016,33
115713,0,0.697575,19,8.933005,10.460844,-0.610191,258,83,0.062016,33
1923,0,0.657661,61,10.057367,11.3972,-0.597056,258,83,0.062016,33
…,…,…,…,…,…,…,…,…,…,…
142969,149997,0.10444,24,6.919684,8.373554,-0.550485,284,117,0.052817,117
51243,149997,0.103798,49,8.25062,9.622119,-0.577156,284,117,0.052817,117
171377,149997,0.103651,32,4.70048,6.287859,-0.655738,284,117,0.052817,117


In [41]:
test_data = Pool(test_data_for_boosting.select(feature_cols).to_pandas(), cat_features=cat_cols, group_id=test_data_for_boosting["cookie"].to_list())

In [47]:
df_pred_catboost = (
    pl.DataFrame(
        {
            "node": test_data_for_boosting["node"],
            "cookie": test_data_for_boosting["cookie"],
            "catboost_score": model.predict(test_data),
        }
    )
    .sort(by=["cookie", "catboost_score"], descending=[False, True])
    .group_by("cookie")
    .head(100)
)

In [48]:
# df_pred_catboost[["cookie", "node"]].write_csv("../data/predict/baseline_als_catboost_v1.0.csv")

In [49]:
# sorted(df_pred_catboost["cookie"].unique().to_list()) == sorted(test_users)

In [50]:
df_valid_eval_reranker

cookie,node,event
i64,u32,i64
62567,214290,10
125487,31674,14
67774,290456,10
124555,222392,15
128032,214262,10
…,…,…
18916,237525,10
30705,303222,13
46155,271546,10


In [51]:
test_als_results = evaluate_models(
    df_valid_eval_reranker, test_als_pred, df_pred_catboost, k_values=[10, 20, 30, 40, 60, 70, 100]
)
print("\nРезультаты ALS модели на тестовом наборе:")
for metric, value in test_als_results.items():
    print(f"{metric}: {value:.4f}")


Результаты ALS модели на тестовом наборе:
ALS_recall@10: 0.0679
CatBoost_recall@10: 0.0737
ALS_recall@20: 0.1035
CatBoost_recall@20: 0.1129
ALS_recall@30: 0.1297
CatBoost_recall@30: 0.1429
ALS_recall@40: 0.1519
CatBoost_recall@40: 0.1672
ALS_recall@60: 0.1872
CatBoost_recall@60: 0.2044
ALS_recall@70: 0.2029
CatBoost_recall@70: 0.2195
ALS_recall@100: 0.2419
CatBoost_recall@100: 0.2575
